In [13]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer
import shutil 

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
files = sorted(glob('output/*.json'), key = lambda x: int(x.split('-')[1].replace('.json', '')))
len(files)

30639

In [4]:
import mp

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            data[i]['filename'] = f
            data[i]['i'] = i
            a = np.array(tokenizer.encode(data[i]['predict_ms'], add_special_tokens = False))
            a = a[a != 50257]
            data[i]['predict_ms'] = tokenizer.decode(a.tolist() + [50257])
            a = np.array(tokenizer.encode(data[i]['predict_en'], add_special_tokens = False))
            a = a[a != 50257]
            data[i]['predict_en'] = tokenizer.decode(a.tolist() + [50257])
            data[i]['audio_filename'] = os.path.join('output-audio', f'{f_split}-{i}.mp3')
        results.extend(data)
    return results

In [7]:
results = loop((files[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 13.06it/s]


In [8]:
len(results)

520

In [9]:
!mkdir sample-set

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
for r in tqdm(results):
    shutil.copyfile(r['audio_filename'], r['audio_filename'].replace('output-audio/', 'sample-set/')) 

100%|██████████| 520/520 [00:00<00:00, 539.89it/s]


In [20]:
# !zip -r sample-set.zip sample-set

In [22]:
with open('sample-set.jsonl', 'w') as fopen:
    for r in tqdm(results):
        r['audio_filename'] = r['audio_filename'].replace('output-audio/', 'sample-set/')
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 520/520 [00:00<00:00, 80842.07it/s]


In [23]:
from huggingface_hub import HfApi
api = HfApi()

In [24]:
api.upload_file(
    path_or_fileobj='sample-set.jsonl',
    path_in_repo='sample-set.jsonl',
    repo_id='mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3/blob/main/sample-set.jsonl'

In [25]:
api.upload_file(
    path_or_fileobj='sample-set.zip',
    path_in_repo='sample-set.zip',
    repo_id='mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3',
    repo_type='dataset',
)

sample-set.zip:   0%|          | 0.00/82.3M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3/blob/main/sample-set.zip'